### Q1 – Discretisation and numerical solution (20×20 grid)

We lossen
$$
-\left(\frac{\partial^2\psi}{\partial x^2} + \frac{\partial^2\psi}{\partial y^2}\right) = E \psi(x,y),
\quad 0<x<1,\;0<y<1,
\quad \psi = 0 \text{ op de randen}
$$
numeriek op in een doos van \(1\times 1\).

#### Rooster en differentieschema

- We nemen \(N = 20\) **inwendige** roosterpunten in elke richting.  
- Roosterstap:
  $$
  h = \frac{1}{N+1}.
  $$
- Roosterpunten:
  $$
  x_i = i h,\quad y_j = j h,\quad i,j = 1,\dots,N.
  $$

Gecentreerde tweede afgeleiden op \((x_i,y_j)\):

$$
\frac{\partial^2 \psi}{\partial x^2}(x_i,y_j)
\approx
\frac{\psi_{i+1,j} - 2\psi_{i,j} + \psi_{i-1,j}}{h^2},
$$

$$
\frac{\partial^2 \psi}{\partial y^2}(x_i,y_j)
\approx
\frac{\psi_{i,j+1} - 2\psi_{i,j} + \psi_{i,j-1}}{h^2}.
$$

Ingevuld in de vergelijking geeft:

$$
\frac{4}{h^2}\psi_{i,j}
-\frac{1}{h^2}\big(
\psi_{i+1,j} + \psi_{i-1,j} + \psi_{i,j+1} + \psi_{i,j-1}
\big)
= E \psi_{i,j},
$$

met \(\psi=0\) buiten het rooster (Dirichlet-randvoorwaarden).

#### Matrixvorm

We stoppen alle \(\psi_{i,j}\) in één vector \(\Psi\) van lengte \(N^2\).  
De vergelijking wordt dan een matrix-eigenwaardeprobleem:
$$
A \Psi = E \Psi,
$$
waarbij $A$ de 2D-discrete Laplace-operator $-\nabla^2$ is (5-punts stencil).

In 1D is de matrix voor \(-\frac{d^2}{dx^2}\):

$$
T = \frac{1}{h^2}
\begin{pmatrix}
2 & -1 & 0 & \dots  & 0 \\
-1 & 2 & -1 & \dots & 0 \\
0 & -1 & 2 & \dots  & 0 \\
\vdots & \vdots & \vdots & \ddots & -1 \\
0 & 0 & 0 & -1 & 2
\end{pmatrix}_{N\times N},
$$

en in 2D:

$$
A = I_N \otimes T \;+\; T \otimes I_N.
$$

#### Python-implementatie (eigenwaarden, eigenvectoren en kansdichtheid)

```python
import numpy as np
from scipy.sparse import diags, kron, identity
from scipy.sparse.linalg import eigsh

# Parameters
N = 20          # number of interior points in x and y
L = 1.0
h = L / (N + 1) # grid spacing

# 1D operator T for -d^2/dx^2 with Dirichlet BC
main = 2.0 * np.ones(N)
off  = -1.0 * np.ones(N - 1)

T = diags([off, main, off], offsets=[-1, 0, 1]) / h**2
I = identity(N)

# 2D operator A = -∇^2
A = kron(I, T) + kron(T, I)  # size (N^2, N^2)

# Compute a few lowest eigenvalues/eigenvectors
num_eigs = 9
E, vecs = eigsh(A, k=num_eigs, which="SM")  # smallest-magnitude eigenvalues

# Sort them
idx = np.argsort(E)
E = E[idx]
vecs = vecs[:, idx]

print("Lowest energies:", E)


### Q1 – Discretising and solving the 2D TISE in a 1×1 box

We solve
$$
-\left(\frac{\partial^2\psi}{\partial x^2} + \frac{\partial^2\psi}{\partial y^2}\right)
= E\,\psi(x,y),
\quad 0<x<1,\;0<y<1,
$$
with infinite potential outside the box, so
$$
\psi(x,y) = 0 \text{ op de randen}.
$$

#### Discretisation (20×20 grid)

- We take \(N = 20\) **interior** grid points in both \(x\) and \(y\).
- Grid spacing:
  $$
  h = \frac{1}{N+1}.
  $$
- Grid points:
  $$
  x_i = i h,\quad y_j = j h,\quad i,j = 1,\dots,N.
  $$

Using centered finite differences, the second derivatives at \((x_i,y_j)\) are approximated by
$$
\frac{\partial^2 \psi}{\partial x^2}(x_i,y_j)
\approx
\frac{\psi_{i+1,j} - 2\psi_{i,j} + \psi_{i-1,j}}{h^2},
$$
$$
\frac{\partial^2 \psi}{\partial y^2}(x_i,y_j)
\approx
\frac{\psi_{i,j+1} - 2\psi_{i,j} + \psi_{i,j-1}}{h^2}.
$$

The discrete equation becomes
$$
\frac{4}{h^2}\psi_{i,j}
-\frac{1}{h^2}\big(
\psi_{i+1,j} + \psi_{i-1,j} + \psi_{i,j+1} + \psi_{i,j-1}
\big)
= E\,\psi_{i,j},
$$
with$\psi=0$ outside the grid (Dirichlet boundary conditions).

#### Matrix form and numerical solution

We collect all $\psi_{i,j}$ into a vector $\Psi$ of length $N^2$.  
The equation can be written as
$$
H \Psi = E \Psi,
$$
where $H$ is the discrete version of $-\nabla^2$ (5-point stencil).  
Using `scipy.sparse` we construct
- a 1D operator $T$ for $-\mathrm{d}^2/\mathrm{d}x^2$,
- then build the 2D operator as
  $$
  H = I \otimes T + T \otimes I.
  $$

With `scipy.sparse.linalg.eigsh` we compute the lowest eigenvalues $E_n$ and eigenvectors $\Psi_n$.

#### Probability density

Each eigenvector $\Psi_n$ is reshaped to a 2D grid $\psi_n(x_i,y_j)$ and normalised such that
$$
\sum_{i,j} |\psi_n(x_i,y_j)|^2\,h^2 \approx 1.
$$

The **probability density** for state $n$ is then
$$
\rho_n(x_i,y_j) = |\psi_n(x_i,y_j)|^2 = \psi_n(x_i,y_j)\,\psi_n^*(x_i,y_j).
$$

We can visualise $\rho_n$ as a heatmap to see where the particle is most likely to be found.


### Step 2 – Exact analytical solution

For the infinite 2D box $0 < x < 1,\; 0 < y < 1$ with $\psi=0$ on the boundaries, the exact eigenvalues and eigenfunctions of
$$
-\nabla^2 \psi(x,y) = E \psi(x,y)
$$
are
$$
E_{k,j} = (k^2 + j^2)\,\pi^2,
$$
$$
\psi_{k,j}(x,y) = \sin(k \pi x)\,\sin(j \pi y),
$$
for $k,j = 1,2,3,\dots$.

These $\psi_{k,j}$ solve the eigenvalue equation exactly (up to an overall normalisation factor, which does not change the eigenvalue problem).

For later comparison with the numerical solution, we implement:

- a function that returns $E_{k,j}$,
- a function that returns $\psi_{k,j}(x,y)$ for given $k,j$ and coordinates $(x,y)$, where $x$ and $y$ can be scalars or NumPy arrays.


### Q3 – Vergelijking van numerieke en analytische eigenwaarden

In Q3 moet je de **numerieke eigenwaarden** uit Q1 vergelijken met de **exacte analytische eigenwaarden** uit Q2.

Voor de oneindige 2D-doos zijn de exacte energieën:
$$
E_{k,j} = (k^2 + j^2)\,\pi^2, \quad k,j = 1,2,\dots
$$

**Plan:**

1. Bereken de laagste $N_\text{eig} = 100$ numerieke eigenwaarden van de matrix $A$ (de gediscretiseerde $-\nabla^2$).
2. Sorteer ze en noteer ze als $E_n^\text{num}$ met $n = 1,\dots,100$.
3. Genereer alle exacte $E_{k,j}$ voor $k,j = 1,\dots,N$ (hier $N=20$), flatten naar één lijst en sorteer: $E_n^\text{exact}$.
4. Plot $E_n^\text{num}$ en $E_n^\text{exact}$ in één figuur als functie van de index $n$.
5. Zoom in op de eerste 100 waarden (x-as: $n=1,\dots,100$).

Zo zie je direct hoe goed de numerieke spectrum de analytische oplossing benadert, en vanaf waar de discretisatiefouten beginnen te domineren.

---

#### Code voor de Q3-codecel

Deze code gaat ervan uit dat:

- je in Q1 de matrix `A` en de gridgrootte `N` al hebt gedefinieerd,
- je in Q2 een functie `exact_energy(k, j)` hebt gedefinieerd die $E_{k,j}$ teruggeeft.

```python
# Q3: compare numerical and analytical eigenvalues

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh

# aantal eigenwaarden dat we willen vergelijken
num_eigs_q3 = 100

# --- numerieke eigenwaarden (Matrix A komt uit Q1) ---
E_num, _ = eigsh(A, k=num_eigs_q3, which="SM")  # smallest magnitude eigenvalues
E_num = np.sort(E_num)

# --- analytische eigenwaarden E_{k,j} = (k^2 + j^2) * pi^2 ---
E_exact_all = []
for k in range(1, N + 1):
    for j in range(1, N + 1):
        E_exact_all.append(exact_energy(k, j))   # uit Q2

E_exact_all = np.sort(np.array(E_exact_all))
E_exact = E_exact_all[:num_eigs_q3]             # eerste 100 exacte waarden

# index n = 1, ..., 100
n = np.arange(1, num_eigs_q3 + 1)

# --- plot ---
plt.figure(figsize=(6, 4))

plt.plot(n, E_exact, "o", markersize=4, label="Exact", alpha=0.8)
plt.plot(n, E_num, "x", markersize=4, label="Numerical (FD)", alpha=0.8)

plt.xlabel("Index $n$")
plt.ylabel("Eigenvalue $E_n$")
plt.title("Comparison of numerical and analytical eigenvalues")
plt.legend()
plt.grid(True, alpha=0.3)

# Zoom in on the first 100 values (x-as is sowieso 1..100, maar expliciet zetten kan geen kwaad)
plt.xlim(1, num_eigs_q3)

plt.tight_layout()
plt.show()
